In [5]:
import os
import numpy as np

# Set the input directory containing the .npy and .txt files
input_directory = '/mnt/data/'
# Create a new directory for the arranged .npy files
output_directory = os.path.join(input_directory, 'arranged')
os.makedirs(output_directory, exist_ok=True)

# Define the desired channel order
desired_order = [
    "Fp1", "F3", "C3", "P3", "1", "F7", "T3", "T5", "Fc1",
    "Fc5", "Cp1", "Cp5", "F9", "Fz", "Cz", "Pz", "Fp2", 
    "F4", "C4", "P4", "O2", "F8", "T4", "T6", "Fc2", 
    "Fc6", "Cp2", "Cp6", "F10"
]

# Function to get channel order from a .txt file
def get_channel_order_from_txt(txt_file):
    with open(txt_file, 'r') as f:
        lines = f.readlines()
    channels = []
    for line in lines:
        if "Channel" in line:
            channel_name = line.split(":")[-1].strip()
            channels.append(channel_name)
    return channels

# Process each npy file and arrange channels
for filename in os.listdir(input_directory):
    if filename.endswith('.npy'):
        # Load the .npy file
        npy_file_path = os.path.join(input_directory, filename)
        data = np.load(npy_file_path)

        # Find the corresponding .txt file
        txt_file = filename.replace('_segmented.npy', '') + '.txt'
        txt_file_path = os.path.join(input_directory, txt_file)

        # Get the channel order from the .txt file
        if os.path.exists(txt_file_path):
            channel_order = get_channel_order_from_txt(txt_file_path)

            # Create a mapping from the original indices to the desired order
            index_map = {channel: idx for idx, channel in enumerate(channel_order)}
            arranged_indices = [index_map[channel] for channel in desired_order if channel in index_map]

            # Arrange the channels
            arranged_data = data[:, arranged_indices, :]

            # Save the arranged data in the new directory
            output_file_path = os.path.join(output_directory, filename)
            np.save(output_file_path, arranged_data)